Notebook para generación de un gold dataset de navegación de usuarios simulando el envío desde google Analithics y el CRM de una empresa

In [2]:
import pandas as pd
from faker import Faker
import random
import uuid
from datetime import datetime, timedelta

# Configuración inicial
fake = Faker('es_ES')
Faker.seed(42)
random.seed(42)

# Constantes de Vancubic
PAGES_MAP = {
    '/': 'Inicio - Vancubic',
    '/nosotros': 'Sobre Nosotros - Vancubic',
    '/camperizaciones/cubic-basic': 'Cubic Basic - Kit Camper',
    '/camperizaciones/cubic-pro': 'Cubic Pro - Módulo Completo',
    '/camperizaciones/accesorios': 'Accesorios Camper',
    '/configurador': 'Configurador de Furgo',
    '/blog/mejores-furgonetas-2024': 'Blog - Mejores Furgonetas',
    '/contacto': 'Contacto',
    '/registro': 'Registro de Usuario',
    '/login': 'Iniciar Sesión',
    '/mi-cuenta': 'Área Privada',
    '/checkout/confirmacion': 'Pedido Confirmado'
}

URLS = list(PAGES_MAP.keys())
SOURCES = ['google', 'direct', 'facebook', 'instagram', 'newsletter']
MEDIUMS = ['organic', 'none', 'cpc', 'social', 'email']

---------------------------------------------------------
1. GENERACIÓN DE USUARIOS (users.csv)
---------------------------------------------------------

In [3]:
print("Generando usuarios...")
NUM_USERS = 100
users_data = []

# Rango de fechas de registro (último año)
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

for i in range(NUM_USERS):
    uid = f"U-{str(uuid.uuid4())[:8]}" # User ID del CRM
    pseudo_id = f"GA-{str(uuid.uuid4())}" # GA4 Client ID (Cookie)
    
    reg_date = fake.date_time_between(start_date=start_date, end_date=end_date)
    
    users_data.append({
        'user_id': uid,
        'user_pseudo_id': pseudo_id, # Guardamos esto para vincular sesiones
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'email': fake.email(),
        'city': fake.city(),
        'registration_date': reg_date,
        'device_category': random.choice(['mobile', 'desktop', 'tablet'])
    })

df_users = pd.DataFrame(users_data)

# Guardamos solo lo que iría en un CRM (sin el pseudo_id a veces, 
# pero para el ejercicio lo dejamos o usamos el pseudo_id como clave de cruce "conocida" internamente)
# Para el CSV de usuarios, exportamos los datos de perfil.
df_users_export = df_users[['user_id', 'first_name', 'last_name', 'email', 'city', 'registration_date']]

Generando usuarios...


---------------------------------------------------------
2. GENERACIÓN DE SESIONES (sessions.csv)
---------------------------------------------------------

In [4]:
print("Generando sesiones...")
TARGET_SESSIONS = 500
sessions_data = []

# Seleccionamos a los 20 usuarios que tendrán historial anónimo previo ("Ghost Users")
special_users_indices = random.sample(range(NUM_USERS), 20)

# Generar sesiones para usuarios registrados
for i in range(NUM_USERS):
    user = users_data[i]
    is_special = i in special_users_indices
    
    # Número de sesiones por usuario (entre 1 y 4)
    num_user_sessions = random.randint(1, 4)
    
    # Si es especial, forzamos al menos una sesión ANTES del registro
    if is_special:
        # Sesión Pre-registro (Anónima pero mismo dispositivo/pseudo_id)
        session_date = user['registration_date'] - timedelta(days=random.randint(1, 30))
        session_id = f"S-{str(uuid.uuid4())[:12]}"
        
        sessions_data.append({
            'session_id': session_id,
            'user_pseudo_id': user['user_pseudo_id'],
            'user_id': None, # Importante: Aún no estaba logueado
            'session_date': session_date,
            'source': random.choice(SOURCES),
            'medium': random.choice(MEDIUMS),
            'device': user['device_category'],
            'is_anonymous_history': True # Flag de ayuda (opcional)
        })
    
    # Sesiones Post-registro (Logueadas)
    for _ in range(num_user_sessions):
        # Fecha posterior al registro
        days_after = random.randint(0, 60)
        session_date = user['registration_date'] + timedelta(days=days_after)
        if session_date > datetime.now(): continue # Evitar futuro
        
        session_id = f"S-{str(uuid.uuid4())[:12]}"
        
        sessions_data.append({
            'session_id': session_id,
            'user_pseudo_id': user['user_pseudo_id'],
            'user_id': user['user_id'], # Aquí SI hay ID
            'session_date': session_date,
            'source': random.choice(SOURCES),
            'medium': random.choice(MEDIUMS),
            'device': user['device_category'],
            'is_anonymous_history': False
        })

# Rellenar el resto hasta 500 con tráfico puramente anónimo (gente que nunca se registró)
current_sessions = len(sessions_data)
remaining = TARGET_SESSIONS - current_sessions

for _ in range(remaining):
    anon_pseudo_id = f"GA-{str(uuid.uuid4())}" # Cookie nueva
    session_date = fake.date_time_between(start_date=start_date, end_date=end_date)
    session_id = f"S-{str(uuid.uuid4())[:12]}"
    
    sessions_data.append({
        'session_id': session_id,
        'user_pseudo_id': anon_pseudo_id,
        'user_id': None,
        'session_date': session_date,
        'source': random.choice(SOURCES),
        'medium': random.choice(MEDIUMS),
        'device': random.choice(['mobile', 'desktop']),
        'is_anonymous_history': False
    })

df_sessions = pd.DataFrame(sessions_data).sort_values('session_date')

Generando sesiones...


---------------------------------------------------------
3. GENERACIÓN DE EVENTOS / PAGEVIEWS (events.csv)
---------------------------------------------------------

In [5]:
print("Generando eventos (pageviews)...")
events_data = []

for _, session in df_sessions.iterrows():
    # Número de páginas vistas en esta sesión
    views_count = random.randint(2, 6)
    
    # Simular navegación
    current_time = session['session_date']
    
    # Flujo simple de navegación
    nav_flow = []
    
    # Entrada
    nav_flow.append(random.choice(['/', '/camperizaciones/cubic-basic', '/blog/mejores-furgonetas-2024']))
    
    # Navegación intermedia
    for _ in range(views_count - 1):
        nav_flow.append(random.choice(URLS))
        
    # Si tiene user_id (está logueado), probabilidad alta de visitar 'mi-cuenta'
    if session['user_id'] and random.random() > 0.5:
        nav_flow.append('/mi-cuenta')

    for page_url in nav_flow:
        event_id = str(uuid.uuid4())
        page_title = PAGES_MAP.get(page_url, 'Vancubic Page')
        
        events_data.append({
            'event_id': event_id,
            'event_name': 'page_view',
            'event_timestamp': current_time,
            'user_pseudo_id': session['user_pseudo_id'],
            'user_id': session['user_id'], # GA4 envía esto si el user está set
            'session_id': session['session_id'],
            'page_location': f"https://vancubic.com{page_url}",
            'page_title': page_title
        })
        
        # Incrementar tiempo entre páginas (30s a 3 min)
        current_time += timedelta(seconds=random.randint(30, 180))

df_events = pd.DataFrame(events_data)

Generando eventos (pageviews)...


---------------------------------------------------------
4. EXPORTACIÓN
---------------------------------------------------------

In [7]:
print("Exportando CSVs...")

# Users: Simula base de datos SQL de clientes
df_users_export.to_csv('../data/users.csv', index=False)

# Sessions: Simula tabla de sesiones de GA4 (flat)
# Mantenemos user_pseudo_id porque es la clave para el grafo
df_sessions.to_csv('../data/sessions.csv', index=False, columns=['session_id', 'user_pseudo_id', 'user_id', 'session_date', 'source', 'medium', 'device'])

# Events: Simula tabla de eventos raw
df_events.to_csv('../data/events.csv', index=False)

print(f"Generación completa:")
print(f"- users.csv: {len(df_users_export)} registros")
print(f"- sessions.csv: {len(df_sessions)} registros")
print(f"- events.csv: {len(df_events)} registros")
print("\nNota: Busca los 20 usuarios que tienen historial anónimo previo.")
print("Pista: Filtra en sessions.csv donde user_id es NULL y luego busca ese user_pseudo_id en otras sesiones donde user_id NO es NULL.")

Exportando CSVs...
Generación completa:
- users.csv: 100 registros
- sessions.csv: 500 registros
- events.csv: 2188 registros

Nota: Busca los 20 usuarios que tienen historial anónimo previo.
Pista: Filtra en sessions.csv donde user_id es NULL y luego busca ese user_pseudo_id en otras sesiones donde user_id NO es NULL.
